# Final cube analysis

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib notebook
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets


import pickle

name_data_file = 'newExtrapolated_allCorrection.pickle'

with open(name_data_file, "rb") as input_file:
    data = pickle.load(input_file)

data.keys()

dict_keys(['geoCUBE', 'phis', 'smoCube', 'gams', 'dipCUBE', 'potCube', 'thes', 'kinCube'])

## Coordinates

In [4]:
from quantumpropagator import fromLabelsToFloats, labTranformA

phis_ext = labTranformA(data['phis'])
gams_ext = labTranformA(data['gams'])
thes_ext = labTranformA(data['thes'])

phiV_ext, gamV_ext, theV_ext = fromLabelsToFloats(data)

# take step
dphi = phis_ext[0] - phis_ext[1]
dgam = gams_ext[0] - gams_ext[1]
dthe = thes_ext[0] - thes_ext[1]

# take range
range_phi = phis_ext[-1] - phis_ext[0]
range_gam = gams_ext[-1] - gams_ext[0]
range_the = thes_ext[-1] - thes_ext[0]


header = '         Labels extr.        internal extr.     dq      range\n'
string = 'Phi -> {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f}\nGam -> {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f}\nThe -> {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f}'
out = (header + string).format(phiV_ext[-1],phiV_ext[0],phis_ext[-1],phis_ext[0],dphi,range_phi,
                               gamV_ext[-1],gamV_ext[0],gams_ext[-1],gams_ext[0],dgam,range_gam,
                               theV_ext[-1],theV_ext[0],thes_ext[-1],thes_ext[0],dthe,range_the)
print(out)

         Labels extr.        internal extr.     dq      range
Phi ->   0.1250  -0.1450  12.5000 -14.5000  -0.5000  27.0000
Gam ->   0.4294   0.1344  24.6050   7.7030  -0.3070  16.9020
The ->   0.5806   1.2008  66.5340 137.6000   0.4470 -71.0660


## NACS

In [5]:
nacs = data['smoCube']

In [6]:
# take out zeros
NACS = nacs[15:-15,15:-15,30:-30]
phis = phis_ext[15:-15]
gams = gams_ext[15:-15]
thes = thes_ext[30:-30]
phiV = phiV_ext[15:-15]
gamV = gamV_ext[15:-15]
theV = theV_ext[30:-30]
# select the two states
NACS.shape

(25, 26, 100, 8, 8, 3)

In [8]:
# x = gam_cut
# y = the_cut
# z = NACS[14,:,:,0,1,0]

# print(x.shape,y.shape,z.shape)

# do3dplot3(x,y,z)

def do3dplot(xs,ys,zss):
    'with mesh function'
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    X,Y = np.meshgrid(ys,xs)
    #ax.plot_wireframe(X, Y, zss)
    #ax.set_zlim(-1, 1)
    ax.scatter(X, Y, zss)
    fig.tight_layout()

def visualize_this_thing(thing,state1,state2,cart,kind,dim):
    along = ['X','Y','Z']
    print('NAC between state ({},{}) along {}\nDoing cut in {} with value ({:8.4f},{:8.4f})'.format(state1,
                                                                state2,
                                                                along[cart],
                                                                kind,
                                                                dimV[kind][dim],
                                                                dims[kind][dim]))
    if kind == 'Phi':
        pot = thing[dim,:,:,state1,state2,cart]
        do3dplot(gams,thes,pot)
    elif kind == 'Gam':
        pot = thing[:,dim,:,state1,state2,cart]
        do3dplot(phis,thes,pot)
    elif kind == 'The':
        pot = thing[:,:,dim,state1,state2,cart]
        do3dplot(phis,gams,pot)


dimV = { 'Phi': phiV, 'Gam': gamV, 'The': theV } # real values
dims = { 'Phi': phis, 'Gam': gams, 'The': thes } # for labels

kinds = ['Phi','Gam','The']

def fun_pot2D(kind,state1, state2, cart,dim):
    visualize_this_thing(NACS, state1, state2, cart, kind, dim)
    
def nested(kinds):
    dimensionV = dimV[kinds]
    interact(fun_pot2D, kind=fixed(kinds), state1 = widgets.IntSlider(min=0,max=7,step=1,value=0), state2 = widgets.IntSlider(min=0,max=7,step=1,value=0), cart = widgets.IntSlider(min=0,max=2,step=1,value=0), dim = widgets.IntSlider(min=0,max=(len(dimensionV)-1),step=1,value=0))

interact(nested, kinds = ['Phi','Gam','The']);

A Jupyter Widget

In [10]:
data['geoCUBE'].shape

(55, 56, 160, 15, 3)

In [11]:
data['geoCUBE']

array([[ 0.        ,  0.        ,  1.078168  ],
       [ 0.        , -1.116359  ,  0.        ],
       [ 0.        ,  1.116359  ,  0.        ],
       [ 0.894773  ,  0.        ,  1.698894  ],
       [-0.894773  ,  0.        ,  1.698894  ],
       [ 0.        , -2.148889  ,  0.336566  ],
       [ 0.        ,  2.148889  ,  0.336566  ],
       [-0.01347234,  1.10841512, -1.54092063],
       [ 0.01347234,  1.10841512, -1.54092063],
       [ 0.01347234, -1.10841512, -1.54092063],
       [-0.01347234, -1.10841512, -1.54092063],
       [-0.70839334,  1.77011512, -2.03512663],
       [ 0.70839334,  1.77011512, -2.03512663],
       [ 0.70839334, -1.77011512, -2.03512663],
       [-0.70839334, -1.77011512, -2.03512663]])

In [27]:
pot = data['potCube'] - data['potCube'].min()
A = pot
np.unravel_index(A.argmin(), A.shape)

(29, 28, 55, 0)

In [28]:
pot[29,28,55]

array([ 0.        ,  0.2056204 ,  0.22117185,  0.22386865,  0.23787536,
        0.2553807 ,  0.26715192,  0.27743809])